In [1]:
import numpy as np
import pickle
import pathlib

from os.path import join
path_to_file = str(pathlib.Path().resolve())
dir_path = join(path_to_file, "../../")
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append(join(dir_path, "HelperFiles"))
from load_data import *


In [2]:
method = "lime"
dataset = "breast_cancer"
K = 3
alpha = 0.1
fname = method + "_" + dataset + "_K" + str(K) + "_fwers"
with open(join(dir_path, "Experiments", "Results", "alpha0.2", fname), "rb") as fp:
        fwers_dict = pickle.load(fp)
fwers = np.array(list(fwers_dict.values()))
np.round(fwers*100)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

- alpha = 0.1, K=2 & 5
    - LIME always 0 FWER except K=5 credit, where it fucks up
    - RankSHAP good: not always 0, few 9/10 controlled
    - KernelSHAP always 0 FWER on K=2; always fails on K=5
- alpha = 0.2, K=3 & 7
    - from table: rankshap & lime good, though lime fails twice on K=7
    - kernelshap: same as alpha=0.1, except also fails on bank when K=3

In [22]:
import os
alpha = 0.1
Ks = [2,5]
# Ks = [3, 7]
# alpha = 0.2
datasets = ["census", "bank", "brca", "breast_cancer", "credit"]

method = "lime"
avg_mat = np.empty((len(Ks), len(datasets)))
control_mat = np.empty((len(Ks), len(datasets)))
for i, K in enumerate(Ks):
    for j, dataset in enumerate(datasets):
        avg_mat[i, j] = np.nan
        control_mat[i, j] = np.nan
        fname = method + "_" + dataset + "_K" + str(K) + "_fwers"
        path = join(dir_path, "Experiments", "Results", "alpha"+str(alpha), fname)
        if os.path.exists(path):
                with open(path, "rb") as fp:
                        fwers_dict = pickle.load(fp)
                if len(fwers_dict) >= 1:## == 10: 
                        fwers = np.array(list(fwers_dict.values()))
                        avg_fwer = np.mean(fwers)
                        avg_mat[i, j] = np.round(avg_fwer*100)
                        prop_fwer_control = np.mean(fwers < alpha)
                        control_mat[i, j] = prop_fwer_control

print(method.upper())
print("Average FWER (%)")
print(avg_mat)
print("Proportion of FWERs below alpha")
print(control_mat)

LIME
Average FWER (%)
[[ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0. nan]]
Proportion of FWERs below alpha
[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1. nan]]


In [9]:
for dataset in datasets:
    X_train, y_train, X_test, y_test, mapping_dict = load_data(join(dir_path, "Experiments", "Data"), dataset)
    N = y_train.shape[0] + y_test.shape[0]
    D = X_train.shape[1] if mapping_dict is None else len(mapping_dict)
    print(dataset)
    print(N, D)


census
32561 12
bank
45211 16
brca
572 20
breast_cancer
569 30
credit
1000 20


Sample size analysis

In [13]:
# Ks = [3, 7]
# alpha = 0.2
Ks = [2,5]
alpha = 0.1
datasets = ["bank", "brca", "breast_cancer", "census", "credit"]

method = "rankshap"
samples_mat = np.empty((len(Ks), len(datasets)))
for i, K in enumerate(Ks):
    for j, dataset in enumerate(datasets):
        samples_mat[i, j] = np.nan
        fname = method + "_" + dataset + "_K" + str(K) + "_samples"
        path = join(dir_path, "Experiments", "Results", "alpha"+str(alpha), fname)
        if os.path.exists(path):
                with open(path, "rb") as fp:
                        samples = pickle.load(fp)
                        print(len(samples))
                        if len(samples)==10:
                            samples_mat[i,j] = np.median(samples)

print(method.upper())
print("Median number of samples")
print(samples_mat)


10
10
10
10
10
10
10
10
10
10
RANKSHAP
Median number of samples
[[ 2159.  13116.   3000.   1452.5  2000. ]
 [47576.  42726.5 13817.5 15117.5 40893. ]]
